In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)|

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

        
        
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch as torch
import torch.nn as nn
from tqdm import tqdm
import tensorflow.keras.losses as losses
#from keras.Model import compile

I had to downgrade tensorflow because of an error I aws getting

In [ ]:
# Import libraries
import keras as keras
#import tensorflow.keras
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import InputLayer, Conv2D, LeakyReLU, Conv2DTranspose, Input, Flatten, Dense
from tensorflow.keras.layers import Reshape, UpSampling2D
from tensorflow.keras.models import Model
#from tensorflow.keras.utils import image_dataset_from_directory, to_categorical
from tensorflow.python.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras import preprocessing
#import shutil
#from keras.layers import InputLayer, Conv2D, LeakyReLU, Conv2DTranspose
from tensorflow.keras.applications.resnet50 import preprocess_input
import tensorflow as tf
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import  to_categorical
from tensorflow.keras.models import Sequential
from pandas import get_dummies
from os import makedirs

In [ ]:
labels = pd.read_csv('/kaggle/input/dog-breed-identification/labels.csv')
sample = pd.read_csv('/kaggle/input/dog-breed-identification/sample_submission.csv')

In [ ]:
train_dir = '/kaggle/input/dog-breed-identification/train/'
test_dir = '/kaggle/input/dog-breed-identification/test/'

In [ ]:
labels.head()
sample.head();
def jpg(id):
    return id+".jpg"

In [ ]:
labels['id'] = labels['id'].apply(lambda x: x + '.jpg')

In [ ]:
classes = sorted(list(set(labels['breed'])))
n_classes = len(classes)  # no of unique breeds
print("Total unique breed: {}".format(n_classes))

# Now to map each label string to an integer label for easy use 
# To do this we create a dict
class_to_num = dict(zip(classes, range(n_classes)))

In [ ]:
breeds_all = labels["breed"]
breed_counts = breeds_all.value_counts()
breed_counts.head()

In [ ]:
CLASS_NAMES = ['scottish_deerhound','maltese_dog','afghan_hound','entlebucher','bernese_mountain_dog']
labels = labels[(labels['breed'].isin(CLASS_NAMES))]
labels = labels.reset_index()
labels.head()

In [ ]:
#LABELS = pd.get_dummies(labels, columns=['breed'])
#LABELS.head()
#warmer but it looks like a bad transform
#LABELS.head()
#I think the flow_from_dataframe does the table conversion so this is unneccessary

In [ ]:
labels.drop('index',axis=1, inplace=True)

It was having memory issues so I cut away alot of the dataset.

In [ ]:
# Distribution of classes in the train dataset
import seaborn as sns
data = labels.breed.value_counts()

plt.figure(figsize=(17,7))
sns.barplot(x=data.index, y=data.values, color='royalblue')
plt.xticks(rotation=90)
plt.show()

## Preprocessing

In [ ]:
classes = sorted(list(set(labels['breed'])))
n_classes = len(classes)  # no of unique breeds
print("Total unique breed: {}".format(n_classes))

# Now to map each label string to an integer label for easy use 
# To do this we create a dict
class_to_num = dict(zip(classes, range(n_classes)))

In [ ]:
input_shape = (224, 224, 3)

In [ ]:
 #maps each image and breed of the image to a numpy array
def images_to_array(directory, label_dataframe, target_size = input_shape):
    
    image_labels = label_dataframe['breed']
    # passes the input shape and all 10222 images from the dataset from the label.csv file
    images = np.zeros([len(label_dataframe), target_size[0], target_size[1], target_size[2]],dtype=np.uint8) 
    y = np.zeros([len(label_dataframe),1],dtype = np.uint8)
    # as we have huge data and limited ram memory. uint8 takes less memory
    
    for ix, image_name in enumerate(label_dataframe['id'].values):
        img_dir = os.path.join(directory, image_name)   # creates directory of each image+ '.jpg')
        img = load_img(img_dir, target_size = target_size)  # reading the image

        images[ix]=img  # saving the image with respect to its id
        del img  # frees memory
        
        dog_breed = image_labels[ix]   # getting breed info for the img
        y[ix] = class_to_num[dog_breed]  # converting breed info to a numerical value 
    
    y = to_categorical(y)  # converting target variable to a one hot encoded matrix     
    
    return images,y

In [ ]:
X_train = []

In [ ]:
X_train.append( images_to_array(train_dir, labels))

In [ ]:
#X_train = images_to_array(train_dir, labels)
#print(X_train)

In [ ]:
datagenerated = ImageDataGenerator(preprocessing_function = preprocess_input,
                      shear_range = 0.2,
                     vertical_flip = True,
                    rescale=1./255,
                     validation_split = 0.1)   

#rescale=1/255.0
#It wasn't able to find images so I tried to get rid of rescale
#was I accidentally rescaling twice...?#

In [ ]:
x_train = datagenerated.flow_from_dataframe(
labels,
directory = '../input/dog-breed-identification/train',
x_col = 'id',
y_col = 'breed',
color_mode = 'rgb',
#class_mode = 'input',
class_mode = 'categorical',
target_size = (224,224),
batch_size = 32,
shuffle =True,
seed = 42,
subset = 'training')

val_ds = datagenerated.flow_from_dataframe(
labels,
directory = '../input/dog-breed-identification/train',
x_col = 'id',
y_col = 'breed',
color_mode = 'rgb',
#class_mode = 'input',
class_mode = 'categorical',
target_size = (224,224),
batch_size = 32,
shuffle =False,
seed = 42,
subset = 'validation')

In [ ]:
#x_train.shape

In [ ]:
#names = train.class_indices
#class_names = [k for k,v in names.items()]
#class_names[:10]


In [ ]:
x,y = next(x_train)
print(y.shape)
print(x.shape)

In [ ]:
import matplotlib.pyplot as plt
def plot(images,labels):
    plt.figure(figsize = [20,15])
    for i in range(30):
        plt.subplot(5,6,i+1)
        plt.imshow(images[i] )
        plt.title(class_names[np.argmax(labels[i])])
        plt.axis('off')

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28*28*3,1500),
            nn.BatchNorm1d(1500),
            nn.ReLU(),
            nn.Linear(1500,1000),
            nn.BatchNorm1d(1000),
            nn.ReLU(),
            nn.Linear(1000, dim_z),
            nn.BatchNorm1d(dim_z),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(dim_z,1000),
            nn.BatchNorm1d(1000),
            nn.ReLU(),
            #nn.Linear(500,1000),
            #nn.ReLU(),
            nn.Linear(1000,1500),
            nn.BatchNorm1d(1500),
            nn.ReLU(),
            nn.Linear(1500,28*28*3)
        )
      
    def encode(self,x):
        return self.encoder(x)
    
    def decode(self,z):
        return self.decoder(z)
        
    def forward(self, x):
        encoded = self.encode(x) 
        decoded = self.decode(encoded)     

        
        return encoded, decoded

In [ ]:
model = Autoencoder()

I delete 'to(DEVICE)' from the previous

In [ ]:
history = train(x_train, val_ds, model_auto, epochs=10, batch_size=64)

In [ ]:
train_loss, val_loss = zip(*history)
plt.figure(figsize=(15,10))
plt.plot(train_loss, label='Train loss')
plt.plot(val_loss, label='Val loss')
plt.legend(loc='best')
plt.xlabel("epochs")
plt.ylabel("loss")
plt.plot();

Notebook Notes

Another Notebook that took me hundreds of iterations.

I learned the distinction between .Flow(), Flow_from_dataframe() and Flow_from_directory(), which are very important.

I need to get more practice with code because I still found myself getting confused about Class, __call__, @staticmethod et cetera

This notebook entailed a very large amount of learning, so, as per usual, the next notebook will have considerably fewer iterations.